In [1]:
%pip install accelerate -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 8.5 MB/s eta 0:00:00


In [2]:
%pip install torchinfo -qq

Note: you may need to restart the kernel to use updated packages.


In [8]:
import accelerate
from diffusers import AutoencoderKL
from diffusers import AsymmetricAutoencoderKL,VQModel
from torchinfo import summary

### Load pretrained model

In [18]:
#vae = AutoencoderKL.from_pretrained("stabilityai/stable-diffusion-2-1", subfolder="vae")
#vae=  AsymmetricAutoencoderKL.from_pretrained("cross-attention/asymmetric-autoencoder-kl-x-1-5")
vae=VQModel.from_pretrained("femboysLover/vqmodel_tf2_spy") 

The config attributes {'force_upcast': False, 'lookup_from_codebook': False, 'mid_block_add_attention': True} were passed to VQModel, but are not expected and will be ignored. Please verify your config.json configuration file.


In [19]:
vae.encoder.conv_in.weight # Check the weights

Parameter containing:
tensor([[[[ 1.3137e-01, -7.6673e-02, -1.2315e-02],
          [-1.7484e-02, -1.5915e-01,  1.6061e-02],
          [-1.2178e-02, -3.7024e-05,  1.9192e-01]],

         [[ 9.1592e-02,  7.9277e-02,  1.1562e-01],
          [-6.0763e-02, -1.5388e-01, -1.8556e-01],
          [-1.7884e-01, -7.6212e-02,  1.2246e-01]],

         [[-1.1030e-01, -8.2834e-02,  1.6165e-01],
          [ 1.1976e-01,  1.3175e-01, -9.9409e-02],
          [ 1.1129e-01, -1.1815e-01, -3.3323e-02]]],


        [[[ 1.3227e-01, -1.8837e-01, -1.5471e-01],
          [ 3.4216e-02,  1.2532e-01,  1.1838e-01],
          [-1.4533e-01,  6.9777e-02, -4.4973e-02]],

         [[ 1.6917e-01, -1.2936e-03, -2.7952e-02],
          [ 5.0875e-02, -1.1623e-01,  1.0134e-01],
          [-1.0846e-01, -5.9241e-02, -1.5291e-01]],

         [[ 1.1013e-01, -1.6808e-03, -3.6087e-03],
          [-1.3841e-01,  7.9144e-02, -9.5102e-03],
          [ 1.2274e-01,  8.7187e-02,  1.5277e-01]]],


        [[[-6.0155e-02, -1.2631e-01,  7.6083

In [13]:
summary(vae, input_size=(4, 3, 256, 256)) # It is pytorch model

Layer (type:depth-idx)                                       Output Shape              Param #
VQModel                                                      [4, 3, 256, 256]          --
├─Encoder: 1-1                                               [4, 1, 32, 32]            --
│    └─Conv2d: 2-1                                           [4, 32, 256, 256]         896
│    └─ModuleList: 2-2                                       --                        --
│    │    └─DownEncoderBlock2D: 3-1                          [4, 32, 128, 128]         83,744
│    │    └─DownEncoderBlock2D: 3-2                          [4, 64, 64, 64]           316,992
│    │    └─DownEncoderBlock2D: 3-3                          [4, 128, 32, 32]          1,264,768
│    │    └─DownEncoderBlock2D: 3-4                          [4, 256, 32, 32]          4,462,592
│    └─UNetMidBlock2D: 2-3                                   [4, 256, 32, 32]          --
│    │    └─ModuleList: 3-7                                  --        

In [4]:
summary(vae, input_size=(4, 3, 256, 256)) # It is pytorch model

Layer (type:depth-idx)                                       Output Shape              Param #
AutoencoderKL                                                [4, 3, 256, 256]          --
├─Encoder: 1-1                                               [4, 8, 32, 32]            --
│    └─Conv2d: 2-1                                           [4, 128, 256, 256]        3,584
│    └─ModuleList: 2-2                                       --                        --
│    │    └─DownEncoderBlock2D: 3-1                          [4, 128, 128, 128]        738,944
│    │    └─DownEncoderBlock2D: 3-2                          [4, 256, 64, 64]          2,690,304
│    │    └─DownEncoderBlock2D: 3-3                          [4, 512, 32, 32]          10,754,560
│    │    └─DownEncoderBlock2D: 3-4                          [4, 512, 32, 32]          9,443,328
│    └─UNetMidBlock2D: 2-3                                   [4, 512, 32, 32]          --
│    │    └─ModuleList: 3-7                                  --  

In [20]:
vae.save_config("pretrain_vqvae_config") # Save the architecture

We can now check the `config.json` file under `test` directory

In [21]:
vae2 = AutoencoderKL()
vae3 = VQModel()# Initialize the default model

In [22]:
vae3.encoder.conv_in.weight # Check the weights

Parameter containing:
tensor([[[[ 0.1571,  0.1370, -0.1284],
          [ 0.1526, -0.1805,  0.0526],
          [ 0.0619, -0.0808,  0.1220]],

         [[-0.0955,  0.1318,  0.0144],
          [-0.0679, -0.1224, -0.0731],
          [ 0.1781, -0.1017,  0.1256]],

         [[ 0.0523,  0.1313,  0.0329],
          [ 0.1741,  0.0002,  0.0890],
          [ 0.1006,  0.1168,  0.0486]]],


        [[[-0.0215, -0.1824,  0.0499],
          [ 0.1732,  0.0363, -0.1204],
          [ 0.1817, -0.1418, -0.1616]],

         [[ 0.0953, -0.0837,  0.0653],
          [-0.1696,  0.0097, -0.1206],
          [ 0.0837, -0.0029,  0.1114]],

         [[ 0.1488, -0.1039, -0.1807],
          [-0.0766, -0.0482, -0.1256],
          [-0.1699, -0.1099, -0.1500]]],


        [[[ 0.0506,  0.0888,  0.1016],
          [ 0.0331,  0.0381,  0.1501],
          [ 0.1849, -0.0009, -0.1119]],

         [[-0.0926,  0.0807,  0.0745],
          [-0.0056, -0.1178,  0.1583],
          [ 0.0800, -0.1027,  0.1357]],

         [[ 0.1067,  0

In [10]:
summary(vae2, input_size=(4, 3, 256, 256)) # It is pytorch model

Layer (type:depth-idx)                                       Output Shape              Param #
AutoencoderKL                                                [4, 3, 256, 256]          --
├─Encoder: 1-1                                               [4, 8, 256, 256]          169,736
│    └─Conv2d: 2-1                                           [4, 64, 256, 256]         1,792
│    └─ModuleList: 2-6                                       --                        (recursive)
│    │    └─DownEncoderBlock2D: 3-1                          [4, 64, 256, 256]         74,112
├─Decoder: 1-12                                              --                        (recursive)
│    └─UNetMidBlock2D: 2-28                                  --                        (recursive)
│    │    └─ModuleList: 3-27                                 --                        (recursive)
├─Encoder: 1-13                                              --                        (recursive)
│    └─ModuleList: 2-6                

In [11]:
vae2.save_config("test2") # Save the architecture

We can now check the `config.json` file under `test2` directory

In [6]:
vae3 = AutoencoderKL.from_config(AutoencoderKL.load_config('pretrain_config')) # Use the architecture from the first one

In [7]:
vae3.encoder.conv_in.weight # Check the weights

Parameter containing:
tensor([[[[-0.1270,  0.1345, -0.0171],
          [-0.1601, -0.0996, -0.0381],
          [-0.1200, -0.0681,  0.0477]],

         [[-0.1885,  0.0969,  0.1885],
          [ 0.1415, -0.0019,  0.0061],
          [-0.1334, -0.1650,  0.0798]],

         [[-0.1388,  0.0151,  0.0285],
          [-0.0226,  0.0959,  0.0583],
          [ 0.0825,  0.1313,  0.1446]]],


        [[[-0.1794,  0.0011, -0.0538],
          [-0.1877,  0.1385, -0.1143],
          [ 0.0861, -0.1858,  0.1744]],

         [[-0.0451,  0.1532, -0.1798],
          [-0.0843,  0.0116,  0.0332],
          [ 0.0835,  0.0745, -0.1181]],

         [[-0.0206, -0.0614, -0.0194],
          [ 0.1137,  0.0359,  0.0383],
          [-0.0318,  0.0256,  0.0550]]],


        [[[ 0.0165, -0.1481, -0.1486],
          [ 0.1308, -0.1867,  0.0899],
          [ 0.0761,  0.1512, -0.1444]],

         [[ 0.0898, -0.0772, -0.0433],
          [-0.0629, -0.0729,  0.1035],
          [ 0.0199, -0.0770, -0.0324]],

         [[-0.0895, -0

In [49]:
summary(vae3, input_size=(4, 3, 256, 256)) # It is pytorch model

Layer (type:depth-idx)                                       Output Shape              Param #
AutoencoderKL                                                [4, 3, 256, 256]          --
├─Encoder: 1-1                                               [4, 8, 32, 32]            33,421,064
│    └─Conv2d: 2-1                                           [4, 128, 256, 256]        3,584
│    └─ModuleList: 2-34                                      --                        (recursive)
│    │    └─DownEncoderBlock2D: 3-1                          [4, 128, 128, 128]        738,944
├─Decoder: 1-40                                              --                        (recursive)
│    └─UNetMidBlock2D: 2-96                                  --                        (recursive)
│    │    └─ModuleList: 3-101                                --                        (recursive)
├─Encoder: 1-41                                              --                        (recursive)
│    └─ModuleList: 2-34           